In [12]:
# pip install langchain --upgrade
# Version: 0.0.164

!pip3 install tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/b8/eb/234646d9eefda8a500d0fd88b05bf625a90ed18054124349db26e558276e/tiktoken-0.5.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/b8/ad/3398312096118c4e62a5827664e52a04d5068e84d04142dd4a0da8a567ae/regex-2023.10.3-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     -------------------------------------  41.0/42.0 kB 487.6 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 406.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/759.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/759.8 kB ? eta -:--:--
   ----- ---------------------------------- 112.6/759.8 kB ? eta -:--:--
   ------- -------------------------------- 143.4/7


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# PDF Loaders. If unstructured gives you a hard time, try PyPDFLoader
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv

load_dotenv()

False

### Load your data

In [3]:
loader = PyPDFLoader("field-guide-to-data-science.pdf")

## Other options for loaders 
# loader = UnstructuredPDFLoader("../data/field-guide-to-data-science.pdf")
# loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")

In [4]:
data = loader.load()

In [5]:
# Note: If you're using PyPDFLoader then it will split by page for you already
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[30].page_content)} characters in your document')

You have 126 document(s) in your data
There are 2812 characters in your document


### Chunk your data up into smaller documents

In [6]:
# Note: If you're using PyPDFLoader then we'll be splitting for the 2nd time.
# This is optional, test out on your own data.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [7]:
print (f'Now you have {len(texts)} documents')

Now you have 258 documents


### Create embeddings of your documents to get ready for semantic search

In [8]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

C:\Users\namus\miniconda3\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Check to see if there is an environment variable with you API keys, if not, use what you put below

In [9]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'YOUR API KEY GOES HERE')

In [10]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

### Option #1: Pinecone
If you want to use pinecone, run the code below, if not then skip over to Chroma below it. You must go to [Pinecone.io](https://www.pinecone.io/) and set up an account

In [13]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY', 'YOUR API KEY GOES HERE')
PINECONE_API_ENV = os.getenv('PINECONE_API_ENV', 'YOUR ENV GOES HERE') # You may need to switch with your env

# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "YOUR index GOES HERE" # put in the name of your pinecone index here

docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)

### Query those docs to get your answer back

In [14]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [15]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [18]:
query = "How long does it take to learn data science?"
docs = docsearch.similarity_search(query)

In [19]:
chain.run(input_documents=docs, question=query)

' It depends on the individual and the level of mastery they are trying to achieve. Data science requires a significant investment of time across a variety of tasks, and multiple techniques are often applied before one yields interesting results.'